In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [2]:
import platform

from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    path = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown system... sorry~~~~')

# 1. 법정동 코드 가공

In [3]:
locCode = pd.read_excel('../data/법정동코드.xlsx')
locCode.head()

,법정동코드,시도명,시군구명,읍면동명,동리명,생성일자,말소일자
0,1100000000,서울특별시,NaN,NaN,NaN,19880423,NaN
1,1111000000,서울특별시,종로구,NaN,NaN,19880423,NaN
2,1111010100,서울특별시,종로구,청운동,NaN,19880423,NaN
3,1111010200,서울특별시,종로구,신교동,NaN,19880423,NaN
4,1111010300,서울특별시,종로구,궁정동,NaN,19880423,NaN


In [4]:
#읍면동이 NaN인 데이터 추출 > 읍면동 제외
locCode = locCode.drop(locCode[locCode['읍면동명'].isna()].index)
locCode.head()

,법정동코드,시도명,시군구명,읍면동명,동리명,생성일자,말소일자
2,1111010100,서울특별시,종로구,청운동,NaN,19880423,NaN
3,1111010200,서울특별시,종로구,신교동,NaN,19880423,NaN
4,1111010300,서울특별시,종로구,궁정동,NaN,19880423,NaN
5,1111010400,서울특별시,종로구,효자동,NaN,19880423,NaN
6,1111010500,서울특별시,종로구,창성동,NaN,19880423,NaN


In [5]:
# 시군구가 NaN이 아닌 데이터 추출 > 시도명 제외


In [6]:
locCode

,법정동코드,시도명,시군구명,읍면동명,동리명,생성일자,말소일자
2,1111010100,서울특별시,종로구,청운동,NaN,19880423,NaN
3,1111010200,서울특별시,종로구,신교동,NaN,19880423,NaN
4,1111010300,서울특별시,종로구,궁정동,NaN,19880423,NaN
5,1111010400,서울특별시,종로구,효자동,NaN,19880423,NaN
6,1111010500,서울특별시,종로구,창성동,NaN,19880423,NaN
...,...,...,...,...,...,...,...
20551,5013032022,제주특별자치도,서귀포시,표선면,하천리,20060701,NaN
20552,5013032023,제주특별자치도,서귀포시,표선면,성읍리,20060701,NaN
20553,5013032024,제주특별자치도,서귀포시,표선면,가시리,20060701,NaN
20554,5013032025,제주특별자치도,서귀포시,표선면,세화리,20060701,NaN


In [7]:
# 법정동 코드 컬럼에서 5자리 추출합니다.
# 예) 서울특별시 종로구 > 11000
locCode['지역코드'] = locCode['법정동코드'].astype('str').str[:5].astype('int64')
locCode.head()

,법정동코드,시도명,시군구명,읍면동명,동리명,생성일자,말소일자,지역코드
2,1111010100,서울특별시,종로구,청운동,NaN,19880423,NaN,11110
3,1111010200,서울특별시,종로구,신교동,NaN,19880423,NaN,11110
4,1111010300,서울특별시,종로구,궁정동,NaN,19880423,NaN,11110
5,1111010400,서울특별시,종로구,효자동,NaN,19880423,NaN,11110
6,1111010500,서울특별시,종로구,창성동,NaN,19880423,NaN,11110


In [8]:
# 시도명이 서울특별시인 데이터만 추출 후 시도명, 시군구명, 지역코드 컬럼만 추출 
loc = locCode[locCode['시도명'] == '서울특별시'][['시도명','시군구명','지역코드']].drop_duplicates()
loc

,시도명,시군구명,지역코드
2,서울특별시,종로구,11110
90,서울특별시,중구,11140
165,서울특별시,용산구,11170
202,서울특별시,성동구,11200
220,서울특별시,광진구,11215
228,서울특별시,동대문구,11230
239,서울특별시,중랑구,11260
246,서울특별시,성북구,11290
286,서울특별시,강북구,11305
291,서울특별시,도봉구,11320


# 2. 실거래가 데이터 가공

In [9]:
# 실거래가 데이터 불러오기  전용면적    3.3나눠서 평으로
apt = pd.read_csv('../data/2019년 서울 아파트 실거래가.csv',encoding='cp949')
apt.head()

,건축년도,거래년도,거래월,거래일자,법정동,거래금액,아파트이름,전용면적,지역코드,층,지번
0,2008,2019,1,7,사직동,"105,000",광화문풍림스페이스본(101동~105동),97.61,11110,10,9
1,2008,2019,1,11,사직동,"162,000",광화문풍림스페이스본(106동),163.33,11110,11,9-1
2,2008,2019,1,19,사직동,"119,000",광화문풍림스페이스본(101동~105동),131.44,11110,13,9
3,2008,2019,1,30,사직동,"119,000",광화문풍림스페이스본(101동~105동),147.31,11110,1,9
4,2004,2019,1,25,내수동,"160,000",경희궁의아침4단지,150.48,11110,14,73


In [10]:
# 데이터 프레임의 컬럼을 확인합니다.
apt.columns

Index(['건축년도', '거래년도', '거래월', '거래일자', '법정동', '거래금액', '아파트이름', '전용면적', '지역코드',
       '층', '지번'],
      dtype='object')

In [11]:
# 데이터 프레임의 크기(차원)을 조회합니다
apt.shape

(68864, 11)

In [12]:
# 불러온 데이터에 NA값이 있는지 조회합니다.
apt.isna().sum()

건축년도     0
거래년도     0
거래월      0
거래일자     0
법정동      0
거래금액     0
아파트이름    0
전용면적     0
지역코드     0
층        0
지번       0
dtype: int64

In [13]:
# 실거래가 데이터프레임 정보 조회
apt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68864 entries, 0 to 68863
Data columns (total 11 columns):
건축년도     68864 non-null int64
거래년도     68864 non-null int64
거래월      68864 non-null int64
거래일자     68864 non-null int64
법정동      68864 non-null object
거래금액     68864 non-null object
아파트이름    68864 non-null object
전용면적     68864 non-null float64
지역코드     68864 non-null int64
층        68864 non-null int64
지번       68864 non-null object
dtypes: float64(1), int64(6), object(4)
memory usage: 5.8+ MB


In [14]:
# 거래금액 컬럼의 콤마(,) 제거 후 데이터 타입 변경
transform = apt['거래금액'].str.split(',').str[0] + apt['거래금액'].str.split(',').str[1]
apt['거래금액'] = transform.astype('int64')

In [15]:
# 실거래가 데이터프레임 정보 조회 (거래금액 컬럼 변경 확인)
apt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68864 entries, 0 to 68863
Data columns (total 11 columns):
건축년도     68864 non-null int64
거래년도     68864 non-null int64
거래월      68864 non-null int64
거래일자     68864 non-null int64
법정동      68864 non-null object
거래금액     68864 non-null int64
아파트이름    68864 non-null object
전용면적     68864 non-null float64
지역코드     68864 non-null int64
층        68864 non-null int64
지번       68864 non-null object
dtypes: float64(1), int64(7), object(3)
memory usage: 5.8+ MB


In [16]:
# 전용면적(제곱미터)를 평으로 변환(나누기 3.306)하여 전용면적(평) 컬럼 생성 후 소숫점 둘쨰자리까지 표시합니다.
apt['전용면적(평)'] = round(apt['전용면적'] / 3.306,2)
apt.head()

,건축년도,거래년도,거래월,거래일자,법정동,거래금액,아파트이름,전용면적,지역코드,층,지번,전용면적(평)
0,2008,2019,1,7,사직동,105000,광화문풍림스페이스본(101동~105동),97.61,11110,10,9,29.53
1,2008,2019,1,11,사직동,162000,광화문풍림스페이스본(106동),163.33,11110,11,9-1,49.40
2,2008,2019,1,19,사직동,119000,광화문풍림스페이스본(101동~105동),131.44,11110,13,9,39.76
3,2008,2019,1,30,사직동,119000,광화문풍림스페이스본(101동~105동),147.31,11110,1,9,44.56
4,2004,2019,1,25,내수동,160000,경희궁의아침4단지,150.48,11110,14,73,45.52


In [17]:
# 전용면적을 범주화 하기 위해 전용면적(평) 컬럼을 정수형으로 바꾼 새로운 컬럼을 생성합니다.
apt['전용면적(평)2'] = apt['전용면적(평)'].astype('int')
apt.head()

,건축년도,거래년도,거래월,거래일자,법정동,거래금액,아파트이름,전용면적,지역코드,층,지번,전용면적(평),전용면적(평)2
0,2008,2019,1,7,사직동,105000,광화문풍림스페이스본(101동~105동),97.61,11110,10,9,29.53,29
1,2008,2019,1,11,사직동,162000,광화문풍림스페이스본(106동),163.33,11110,11,9-1,49.40,49
2,2008,2019,1,19,사직동,119000,광화문풍림스페이스본(101동~105동),131.44,11110,13,9,39.76,39
3,2008,2019,1,30,사직동,119000,광화문풍림스페이스본(101동~105동),147.31,11110,1,9,44.56,44
4,2004,2019,1,25,내수동,160000,경희궁의아침4단지,150.48,11110,14,73,45.52,45


In [18]:
# 아래의 코드를 실행하여 전용면적을 범주화 합니다.
daterange = np.arange(0,(int(max(apt['전용면적(평)2']) / 5) + 2) * 5, 5)
apt['전용면적(평)3'] = pd.cut(apt['전용면적(평)2'], daterange)
apt.head()

,건축년도,거래년도,거래월,거래일자,법정동,거래금액,아파트이름,전용면적,지역코드,층,지번,전용면적(평),전용면적(평)2,전용면적(평)3
0,2008,2019,1,7,사직동,105000,광화문풍림스페이스본(101동~105동),97.61,11110,10,9,29.53,29,"(25, 30]"
1,2008,2019,1,11,사직동,162000,광화문풍림스페이스본(106동),163.33,11110,11,9-1,49.40,49,"(45, 50]"
2,2008,2019,1,19,사직동,119000,광화문풍림스페이스본(101동~105동),131.44,11110,13,9,39.76,39,"(35, 40]"
3,2008,2019,1,30,사직동,119000,광화문풍림스페이스본(101동~105동),147.31,11110,1,9,44.56,44,"(40, 45]"
4,2004,2019,1,25,내수동,160000,경희궁의아침4단지,150.48,11110,14,73,45.52,45,"(40, 45]"


In [19]:
# 범주화된 전용면적의 고유값을 확인합니다.
apt['전용면적(평)3'].unique()

[(25, 30], (45, 50], (35, 40], (40, 45], (0, 5], ..., (65, 70], (55, 60], (60, 65], (80, 85], (75, 80]]
Length: 17
Categories (17, interval[int64]): [(0, 5] < (5, 10] < (10, 15] < (15, 20] ... (65, 70] < (70, 75] < (75, 80] < (80, 85]]

# 3. 데이터 프레임 결합

In [20]:
apt['반기'] = np.NaN
apt.at[apt['거래월'] <= 6,'반기'] = '상반기'
apt.at[apt['거래월'] > 6,'반기'] = '하반기'

apt['분기'] = np.NaN
apt.at[(apt['거래월'] > 0) & (apt['거래월'] < 4),'분기']='1분기'
apt.at[(apt['거래월'] > 3) & (apt['거래월'] < 7),'분기']='2분기'
apt.at[(apt['거래월'] > 6) & (apt['거래월'] < 10),'분기']='3분기'
apt.at[(apt['거래월'] > 9) & (apt['거래월'] < 13),'분기']='4분기'

In [21]:
# apt['반기'] = np.NaN
# apt.set_value(apt['거래월'] <= 6,'반기','상반기')
# apt.set_value(apt['거래월'] > 6,'반기','하반기')

# apt['분기'] = np.NaN
# apt.set_value((apt['거래월'] > 0) & (apt['거래월'] < 4),'분기','1분기')
# apt.set_value((apt['거래월'] > 3) & (apt['거래월'] < 7),'분기','2분기')
# apt.set_value((apt['거래월'] > 6) & (apt['거래월'] < 10),'분기','3분기')
# apt.set_value((apt['거래월'] > 9) & (apt['거래월'] < 13),'분기','4분기')

In [22]:
apt['평당거래금액'] = (apt['거래금액'] / apt['전용면적(평)2']).astype('int')
apt.head()

,건축년도,거래년도,거래월,거래일자,법정동,거래금액,아파트이름,전용면적,지역코드,층,지번,전용면적(평),전용면적(평)2,전용면적(평)3,반기,분기,평당거래금액
0,2008,2019,1,7,사직동,105000,광화문풍림스페이스본(101동~105동),97.61,11110,10,9,29.53,29,"(25, 30]",상반기,1분기,3620
1,2008,2019,1,11,사직동,162000,광화문풍림스페이스본(106동),163.33,11110,11,9-1,49.40,49,"(45, 50]",상반기,1분기,3306
2,2008,2019,1,19,사직동,119000,광화문풍림스페이스본(101동~105동),131.44,11110,13,9,39.76,39,"(35, 40]",상반기,1분기,3051
3,2008,2019,1,30,사직동,119000,광화문풍림스페이스본(101동~105동),147.31,11110,1,9,44.56,44,"(40, 45]",상반기,1분기,2704
4,2004,2019,1,25,내수동,160000,경희궁의아침4단지,150.48,11110,14,73,45.52,45,"(40, 45]",상반기,1분기,3555


In [23]:
# 지역 코드를 기준으로 법정동 코드 데이터 프레임과 아파트 실거래가 데이터 프레임을 결합합니다.
df = apt.merge(loc,on='지역코드',how='left')
df

,건축년도,거래년도,거래월,거래일자,법정동,거래금액,아파트이름,전용면적,지역코드,층,지번,전용면적(평),전용면적(평)2,전용면적(평)3,반기,분기,평당거래금액,시도명,시군구명
0,2008,2019,1,7,사직동,105000,광화문풍림스페이스본(101동~105동),97.61,11110,10,9,29.53,29,"(25, 30]",상반기,1분기,3620,서울특별시,종로구
1,2008,2019,1,11,사직동,162000,광화문풍림스페이스본(106동),163.33,11110,11,9-1,49.40,49,"(45, 50]",상반기,1분기,3306,서울특별시,종로구
2,2008,2019,1,19,사직동,119000,광화문풍림스페이스본(101동~105동),131.44,11110,13,9,39.76,39,"(35, 40]",상반기,1분기,3051,서울특별시,종로구
3,2008,2019,1,30,사직동,119000,광화문풍림스페이스본(101동~105동),147.31,11110,1,9,44.56,44,"(40, 45]",상반기,1분기,2704,서울특별시,종로구
4,2004,2019,1,25,내수동,160000,경희궁의아침4단지,150.48,11110,14,73,45.52,45,"(40, 45]",상반기,1분기,3555,서울특별시,종로구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68859,2013,2019,12,30,천호동,14500,하이브2,13.50,11740,11,164-70,4.08,4,"(0, 5]",하반기,4분기,3625,서울특별시,강동구
68860,2009,2019,12,7,강일동,79750,강일리버파크10단지,84.53,11740,9,695,25.57,25,"(20, 25]",하반기,4분기,3190,서울특별시,강동구
68861,2009,2019,12,9,강일동,77800,강일리버파크2단지,84.74,11740,4,668,25.63,25,"(20, 25]",하반기,4분기,3112,서울특별시,강동구
68862,2009,2019,12,25,강일동,80000,강일리버파크7단지,84.83,11740,3,686,25.66,25,"(20, 25]",하반기,4분기,3200,서울특별시,강동구


In [24]:
# 지역 코드 컬럼을 삭제합니다.
del df['지역코드']

In [25]:
# Datetime 컬럼 추가
df['기간'] = df['거래년도'].astype('str')+'-'+df['거래월'].astype('str')+'-'+df['거래일자'].astype('str')
df['기간'] = pd.to_datetime(df['기간'])

In [26]:
# 전용면적(평)3 컬럼 형변환
df['전용면적(평)3'] = df['전용면적(평)3'].astype('str')

In [27]:
#법정동의 공백 제거
df['법정동'] = df['법정동'].str.strip()

In [28]:
df1 = df.copy()

# 4. Data Analysis & Visualization 

In [29]:
import numpy as np 
import pandas as pd 

from plotly.offline import init_notebook_mode, iplot, plot
import plotly as py
init_notebook_mode(connected=True)
import plotly.graph_objs as go

import matplotlib.pyplot as plt

In [30]:
labels = df1['분기'].value_counts().index
values = df1['분기'].value_counts().values

colors = ['aliceblue',  'aqua', 'aquamarine', 'darkturquoise']

trace = go.Pie(labels=labels, values=values,
               hoverinfo='value', textinfo='label+percent', 
               textfont=dict(size=20),
               marker=dict(colors=colors, 
                           line=dict(color='rgb(100,100,100)', 
                                     width=1)
                          )
              )

fw=go.FigureWidget(data=[trace])
fw.update_layout(margin=dict(t=0, b=0, l=0, r=0))
fw

FigureWidget({
    'data': [{'hoverinfo': 'value',
              'labels': array(['4분기', '3분기', '2분기', '1분기'],…

#### 1 - 2019년 서울의 아파트 분기별 거래량
- 목적 : 어느 분기에 아파트 거래가 많이 발생했는지 파악하기 위함


- 결과 : 1분기에서 4분기로 갈수록 거래가 많이 발생하였다.(1분기보다 4분기에 약 4배정도 많은 거래가 일어남)

----------------------------------------------------------------------------------------------------------------------------------------------------------

In [31]:
# data preparation
date = df1.groupby('기간')[['거래금액']].agg('mean').reset_index()
a = date[date['기간'] <'2019-04']
b = date[(date['기간'] >='2019-04') & (date['기간'] <'2019-07')]
c = date[(date['기간'] >='2019-07') & (date['기간'] <'2019-10')]
d = date[(date['기간'] >='2019-10') & (date['기간'] <='2019-12-31')]

a['분기'] = '1분기'
b['분기'] = '2분기'
c['분기'] = '3분기'
d['분기'] = '4분기'

trace0 = go.Box(
    x=a['거래금액'],
    name = '1분기',
    marker = dict(
    color = 'rgb(12, 12, 140)',
    )
)
trace1 = go.Box(
    x=b['거래금액'],
    name = '2분기',
    marker = dict(
    color = '#FF851B',
    )
)

trace2 = go.Box(
    x=c['거래금액'],
    name = '3분기',
    marker = dict(
        color = '#FF4136',
    )
)
trace3 = go.Box(
    x=d['거래금액'],
    name = '4분기',
    marker = dict(
        color = '#3D9970',
    )
)

layout = go.Layout(barmode='group',
                   title='분기별 거래금액 분포',
                   xaxis=dict(title='거래금액'),
                   yaxis=dict( title='분기'),
                   font=dict(size=16),
                   
                  )

data = [trace0, trace1,trace2, trace3]
fig = go.Figure(data = data, layout = layout)
iplot(fig)

#### 2 - 2019년 분기별 서울 아파트 거래금액 분포
- 목적 : 분기별의 거래가격 분포의 차이를 확인하기 위함


- 결과 : 1번 차트에서 1분기에서 2분기로 가면서 거래량도 많이 증가했지만 위의 차트에서 보니 거래금액도 2분기로 가면서 갑자기 높아졌다.


- 의문점 : 거래가격과 거래량이 동시에 급증한다?

----------------------------------------------------------------------------------------------------------------------------------------------------------

In [32]:
data =  df1.pivot_table(index=['시군구명'],columns=['분기'],values='평당거래금액').reset_index().sort_values('1분기',ascending=False)

trace1 = go.Bar(
                x = data['시군구명'],
                y = data['1분기'],
                name = "1분기",
                marker = dict(color = 'steelblue',
                             line=dict(color='rgb(0,0,0)',width=.5)),
                text = '1분기')

trace2 = go.Bar(
                x = data['시군구명'],
                y = data['2분기'],
                name = "2분기",
                marker = dict(color = 'tan',
                              line=dict(color='rgb(0,0,0)',width=.5)),
                text = '2분기')

trace3 = go.Bar(
                x = data['시군구명'],
                y = data['3분기'],
                name = "3분기",
                marker = dict(color = 'teal',
                             line=dict(color='rgb(0,0,0)',width=.5)),
                text = '3분기')

trace4 = go.Bar(
                x = data['시군구명'],
                y = data['4분기'],
                name = "4분기",
                marker = dict(color = 'khaki',
                              line=dict(color='rgb(0,0,0)',width=.5)),
                text = '4분기')

data = [trace1, trace2, trace3, trace4]
layout = go.Layout(barmode='group',
                   title='분기별 각지역의 평균 거래금액',
                   xaxis=dict(title='지역'),
                   yaxis=dict( title='평균 거래금액'),
                   font=dict(size=16),
                  )

fig = go.Figure(data = data, layout = layout)
iplot(fig)

#### 3 - 분기별 지역에 따른 아파트 거래금액
- 목적 : 분기별 지역의 아파트 거래금액 차이를 파악


- 결과 : 예상대로 용산, 강남, 서초의 가격이 높은 편이고 용산을 제외한 대부분의 지역에서 분기가 지나가면서 거래금액이 상승한다.

----------------------------------------------------------------------------------------------------------------------------------------------------------

In [33]:
data = df1.groupby(['건축년도'])[['거래금액']].agg('count').reset_index()
trace1 = go.Bar(
                x = data['건축년도'],
                y = data['거래금액'],
                marker = dict(color = 'rgba(255, 255, 128, 0.5)',
                              line=dict(color='rgb(0,0,0)',width=1.5)))
layout = go.Layout(template= "ygridoff",title = '건축년도에 따른 평균 거래량' , xaxis = dict(title = '건축년도'), yaxis = dict(title = 'Count'),font=dict(size=16))
fig = go.Figure(data = [trace1], layout = layout)
fig.show()

#### 4 - 아파트 건축년도에 따른 평균 거래량
- 목적 : 어떤 연식의 아파트가 많이 거래되었는지 파악


- 결과 : 1990년대 이후의 아파트들이 많이 거래가 되었다는것을 알 수 있음.

----------------------------------------------------------------------------------------------------------------------------------------------------------

In [34]:
# data = df1[df1['시군구명'] == '강남구']
data = df1.groupby(['건축년도'])[['거래금액']].agg('mean').reset_index()
trace1 = go.Bar(
                x = data['건축년도'],
                y = data['거래금액'],
                marker = dict(color = 'rgba(255, 174, 255, 0.5)',
                              line=dict(color='rgb(0,0,0)',width=1.5)))
layout = go.Layout(template= "ygridoff",title = '건축년도에 따른 평균 거래금액' , xaxis = dict(title = '건축년도'), yaxis = dict(title = '평균 거래금액'),font=dict(size=16))
fig = go.Figure(data = [trace1], layout = layout)
fig.show()

#### 5 - 아파트 건축년도에 따른 평균 거래금액
- 목적 : 아파트의 연식에 따른 가격차이를 파악


- 결과 : 90년대보다 오래된 아파트는 거래된 횟수가 적은데 가격이 높게나왔다.


- 의문점 : 거래가 된 구식 아파트의 소재지는 집값이 비싼 지역일까?

----------------------------------------------------------------------------------------------------------------------------------------------------------

In [35]:
# 구식 아파트의 소재지
labels = df1[df1['건축년도'] <1985]['시군구명'].value_counts().index[:6]
values = df1[df1['건축년도'] <1985]['시군구명'].value_counts().values[:6]

trace = go.Pie(labels=labels, values=values,
               hoverinfo='value', textinfo='label+percent', 
               textfont=dict(size=20),
               marker=dict(colors=['whitesmoke','wheat','turquoise','powderblue','skyblue','salmon'], 
                           line=dict(color='rgb(100,100,100)', 
                                     width=1)
                          )
              )

fw=go.FigureWidget(data=[trace])
fw.update_layout(margin=dict(t=0, b=0, l=0, r=0))
fw

FigureWidget({
    'data': [{'hoverinfo': 'value',
              'labels': array(['강남구', '송파구', '강동구', '서초구', …

#### 6 - 1985년보다 오래된 아파트의 지역에 따른 거래량

- 목적 : 연식이 오래된 아파트의 지역은 집값이 비싼 지역인지 파악


- 결과 : 오래된 아파트가 많은 상위6개의 지역은 강동구를 제외하면 모두 아파트값이 비싼 지역임을 알 수 있다.

----------------------------------------------------------------------------------------------------------------------------------------------------------

In [36]:
data = df1.groupby('전용면적(평)3')['거래금액'].agg(['count']).reset_index().sort_values(by='전용면적(평)3')

trace1 = go.Bar(
                x = data['전용면적(평)3'],
                y = data['count'],
                marker = dict(color = 'deepskyblue',
                             line=dict(color='rgb(0,0,0)',width=.5),
                             ))

data = [trace1]
layout = go.Layout(barmode='overlay',
                   title='평수별 거래량',
                   xaxis=dict(title='평수'),
                   yaxis=dict( title='Count'),
                   font=dict(size=16),
                  )

fig = go.Figure(data = data, layout = layout,)
iplot(fig)

#### 7 - 아파트 평수별 거래량
- 목적 : 어떤 평수의 아파트가 많이 거래되었는지 파악


- 결과 : 15~25 평의 아파트가 많이 거래되었다.

----------------------------------------------------------------------------------------------------------------------------------------------------------

In [37]:
import plotly.express as px
data = df1[df1['전용면적(평)3'].isin(df1['전용면적(평)3'].value_counts().index[:5])]
data = data.groupby(['거래월','전용면적(평)3'])['거래금액'].agg(['count','mean']).sort_values('거래월').reset_index()

data.rename(columns={'count':'평균 거래량'},inplace=True)
fig = px.line(data, x="거래월", y="평균 거래량", color='전용면적(평)3')
iplot(fig)

#### 8 - 서울에서 가장 많이 거래된 평수 TOP5의 월간 평균 거래량 추이
- 목적 : 평수별 시간의 흐름에 따른 평균 평당 거래량 파악


- 결과 : 가장 많이 거래가 되는 15~25평대의 아파트의 거래량의 변동이 심한것을 알 수 있다.


- 의문점 : 나머지 평수는 거래량이 대체로 큰 변화가 없는데 유독 15~25평 사이의 아파트의 거래량의 변동이 심한 이유?

----------------------------------------------------------------------------------------------------------------------------------------------------------

In [38]:
top5 = df1['전용면적(평)3'].value_counts().index[:5]
data = df1[df1['전용면적(평)3'].isin(top5)]
data = data.groupby(['거래월','전용면적(평)3'])['평당거래금액'].agg(['count','mean']).sort_values('거래월').reset_index()
data.rename(columns={'mean':'평균 평당 거래금액'},inplace=True)

fig = px.line(data, x="거래월", y="평균 평당 거래금액", color='전용면적(평)3')
iplot(fig)

#### 9 - 서울에서 가장 많이 거래된 아파트 평수 TOP5의 월간 평균 평당가격 추이
- 목적 : 거래금액에서 평수를 나눈 평당 가격의 평수면적별 가격 차이


- 결과 : 평수가 크더라도 평당 가격은 적은 평수가 더 높은것이 정상이지만 서울시 전체를 대상으로 한 그래프이기 떄문에 오차가 있다. 

----------------------------------------------------------------------------------------------------------------------------------------------------------

In [39]:
top5 = df1['전용면적(평)3'].value_counts().index[:5]

data = df1[df1['전용면적(평)3'].isin(top5)]
data = data.groupby(['거래월','전용면적(평)3'])['거래금액'].agg(['count','mean']).sort_values('거래월').reset_index()
data.rename(columns={'mean':'평균 거래금액'},inplace=True)

fig = px.line(data, x="거래월", y="평균 거래금액", color='전용면적(평)3')
iplot(fig)

#### 10 - 서울에서 가장 많이 팔린 아파트 평수 TOP5의 월간 평균 거래금액 추이
- 목적 : 평수별 시간의 흐름에 따른 평균 거래금액 파악, 평수별 가격차이


- 결과 : 상반기에 가격이 전반적으로 상승하고 하반기에는 유지되는 추세임을 알 수 있고 예상대로 평수가 클수록 가격이 비싸짐을 알 수 있다. + 간격이 비례적


----------------------------------------------------------------------------------------------------------------------------------------------------------

In [40]:
import plotly.express as px

dd = df1.copy()
dd['층구분'] = np.NaN

dd.at[dd['층'] > 16,'층구분'] ='고층'
dd.at[(dd['층'] > 8) & (dd['층'] <= 16),'층구분']='중간층'
dd.at[dd['층'] <= 8,'층구분']='저층'

data = dd.groupby(['기간','층구분'])['거래금액'].agg(['mean','count']).reset_index()

fig = px.scatter(data, x="기간", y="mean",color='층구분',size='count',
                title="기간에 따른 층수별 가격변화와 거래량")
fig.show()

#### 11 - 기간에 따른 층수별 거래금액 변화와 거래량
- 목적 : 층수가 거래금액에 미치는 영향과 어떤 층수대가 많이 거래되는지 파악
- 결과 : 저층(-8)과 중간층(9-16)은 고층(17-)보다 거래금액이 저렴한 편이고 거래량도 많다는 것을 알 수 있다.

----------------------------------------------------------------------------------------------------------------------------------------------------------

### 좋았던점
- 분석한 결과를 시각화 한 후에 거기에서 또 어떤것들이 파생될 수 있는지에 대한 분석적인 사고방식이란 것을 조금 맛보게 되었다.
- 팀원들과 같이 분석을 하다보니 나의 실수를 빠르게 캐치할 수 있어서 좋았다.

### 아쉬운점
- 데이터 도메인 지식이 부족해서 분석을 시작할때에 분석 방향에 대한 고민이 많았고 중간에 잘못된 가설로인해 우왕좌왕 하느라 시간을 많이 낭비했던 것이 아쉬웠다. 
